# The Propeller "Switch-On" Efficiency

This notebook aims to recreate Fig. 1 from [Gibson et al. (2017)](https://arxiv.org/abs/1706.04802) which demonstrates how the smoothness of the transition from non-propellering to propellering states can be controlled via the frequency `n` of a hyperbolic `tan` function. This is a purely mathematical treatment of the "switch-on".

## 1. Module Imports

We import standard libraries `numpy` and `matplotlib`. We also import `odeint` from `scipy` which we will use to solve a set of coupled, first order, ordinary differential equations.

<div class="alert alert-block alert-warning">
<b>Warning:</b> You must remember to run the code cell containing imports first every time you launch a notebook. The best practice is to run each code in order each time so that any dependencies will be present.
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

%matplotlib inline

## 2. Setting Global Variables

A range of physical constants are used across the calculations and we define these here so that they are available to following code cells (after it has been run!). We also define an array of time points to solve the ODEs for.

In [ ]:
# Set Global constants
G = 6.674e-8                    # Gravitational constant - cgs units
c = 3.0e10                      # Speed of light - cm/s
R = 1.0e6                       # Magnetar radius - km
Msol = 1.99e33                  # Solar mass - grams
M = 1.4 * Msol                  # Magnetar mass - grams
I = (4.0 / 5.0) * M * R ** 2.0  # Moment of Inertia
GM = G * M

# Time array to calculate over
t_arr = np.logspace(0.0, 6.0, num=10001, base=10.0)

## 3. Defining a Function to handle the Initial Conditions

This model works in cgs units (distance in cm, mass in grams, time in seconds), however it is more useful for the user to provide larger units common in astrophysics (distance in km, mass in solar masses, time in milliseconds). The function below converts the units of the variables to be solved by `odeint` from user-input to cgs, and in the case of the spin period, this also needs to be converted to an angular frequency $\omega$.

In [ ]:
# Calculate initial values to pass to ODEINT
def init_conds(arr):
    """
Function to calculate the initial conditions to pass to ODEINT by converting units.
Principally, converting initial disc mass from solar masses to grams, and calculating an initial angular frequency from a spin period in milliseconds.

Usage >>> init_conds(arr)
arr : list --> element 1: Initial spin period in milliseconds
               element 2: Initial disc mass in solar masses

Returns a list object --> element 1: Initial disc mass in grams
                          element 2: Initial angular frequency in "per second"
    """
    Mdisc0 = arr[1] * Msol                      # Disc mass
    omega0 = (2.0 * np.pi) / (1.0e-3 * arr[0])  # Angular frequency

    return Mdisc0, omega0

## 4. Defining the ODEs to be solved

The following function will be passed to `odeint`. It takes the initial conditions and calculates the derivative of the variables $\dot{M}_{\rm D}$ and $\dot{\omega}$. `odeint` will then integrate these for each point in the time array.

In [ ]:
# Model to be passed to ODEINT to calculate evolving disc mass and angular frequency
def ODEs(y, t, B, MdiscI, RdiscI, epsilon, delta, n, alpha=0.1, cs7=1.0, k=0.9):
    """
Function to pass to ODEINT which will calculate a disc mass and angular frequency for given time points.

Usage >>> ODEs(y, t, B, MdiscI, RdiscI, epsilon, delta, n)
      y : output of init_conds()
      t : array of time points (t_arr above)
      B : Magnetic Field Strength - 10^15 Gauss (float)
 MdiscI : Initial disc mass - Solar masses (float)
 RdiscI : Disc radius - km (float)
epsilon : timescale ration (float)
  delta : mass ratio (float)
      n : propeller "switch-on" (float)
  alpha : Viscosity prescription (float)
    cs7 : Sound speed - 10^7 cm/s (float)
      k : capping fraction (float)
    """
    # Initial conditions
    Mdisc, omega = y
    
    # Constants
    Rdisc = RdiscI * 1.0e5                 # Disc radius - cm
    tvisc = Rdisc / (alpha * cs7 * 1.0e7)  # Viscous timescale - s
    mu = 1.0e15 * B * R ** 3.0             # Magnetic Dipole Moment
    M0 = delta * MdiscI * Msol             # Global Fallback Mass Budget - g
    tfb = epsilon * tvisc                  # Fallback timescale - s
    
    # Radii - Alfven, Corotation, Light Cylinder
    Rm = mu ** (4.0 / 7.0) * GM ** (-1.0 / 7.0) * (Mdisc / tvisc) ** (-2.0 / 7.0)
    Rc = (GM / omega ** 2.0) ** (2.0 / 3.0)
    Rlc = c / omega
    # Cap the Alfven radius
    if Rm >= k * Rlc:
        Rm = k * Rlc
    
    w = (Rm / Rc) ** (3.0 / 2.0)  # Fastness parameter
    
    bigT = 0.5 * I * omega ** 2.0  # Rotational energy
    modW = (0.6 * M * c ** 2.0 * ((GM / (R * c ** 2.0)) / (1.0 - 0.5 * (GM / (R * c ** 2.0)))))  # Binding energy
    rot_param = bigT / modW  # Rotation parameter
    
    Ndip = (-1.0 * mu ** 2.0 * omega ** 3.0) / (6.0 * c ** 3.0)  # Dipole torque
    
    # Mass flow rates and efficiencies
    eta2 = 0.5 * (1.0 + np.tanh(n * (w - 1.0)))
    eta1 = 1.0 - eta2
    Mdotprop = eta2 * (Mdisc / tvisc)  # Propelled
    Mdotacc = eta1 * (Mdisc / tvisc)   # Accreted
    Mdotfb = (M0 / tfb) * ((t + tfb) / tfb) ** (-5.0 / 3.0)  # Fallback rate
    Mdotdisc = Mdotfb - Mdotprop - Mdotacc  # Mass flow through the disc
    
    if rot_param > 0.27:
        Nacc = 0.0  # Prevents magnetar break-u[
    else:
        # Accretion torque
        if Rm >= R:
            Nacc = (GM * Rm) ** 0.5 * (Mdotacc - Mdotprop)
        else:
            Nacc = (GM * R) ** 0.5 * (Mdotacc - Mdotprop)
    
    omegadot = (Nacc + Ndip) / I  # Angular frequency time derivative
    
    return Mdotdisc, omegadot

## 5. Perform the Calculation and Create the Plot

In the below cell, we set up some constants for the calculation and loop over the desired values of `n` we wish to test. We then solve the equations and plot the results. This requires some re-calculation to recover the desired plotting values from the output of `odeint` which is $M_{\rm D}(t)$ and $\omega (t)$.

In [ ]:
lines = [':', '--', '-']  # Linestyles for plotting

# Constants
B = 1.0         # Magnetic field strength - 10^15 Gauss
P = 1.0         # Initial spin period - milliseconds
MdiscI = 0.1    # Initial disc mass - Solar masses
RdiscI = 100.0  # Disc radius - km
epsilon = 1.0   # Timescale ration
delta = 1.0e-6  # Mass ratio
alpha = 1.0     # Sound speed prescription
cs7 = 1.0       # Sound speed - cm/s

# Set-up
mu = 1.0e15 * B * R ** 3.0             # Magnetic Dipole Moment
Rdisc = RdiscI * 1.0e5                 # Convert radius to cm
tvisc = Rdisc / (alpha * cs7 * 1.0e7)  # Viscous timescale

n_vals = [1.0, 10.0, 50.0]    # n values
y0 = init_conds([P, MdiscI])  # Initial conditions

# Loop over values of n
for i, n in enumerate(n_vals):
    soln = odeint(ODEs, y0, t_arr, args=(B, MdiscI, RdiscI, epsilon, delta, n))

    # Split soln
    Mdisc = soln[:,0]
    omega = soln[:,1]
    
    # Radii
    Rm = mu ** (4.0 / 7.0) * GM ** (-1.0 / 7.0) * (Mdisc / tvisc) ** (-2.0 / 7.0)
    Rc = (GM / omega ** 2.0) ** (1.0 / 3.0)
    Rlc = c / omega
    Rm = np.where(Rm >= 0.9 * Rlc, 0.9 * Rlc, Rm)
    
    w = np.sort((Rm / Rc) ** (3.0 / 2.0))  # Fastness Parameter - sorted to make plotting nicer
    eta2 = 0.5 * (1.0 + np.tanh(n * (w - 1.0)))
    
    # Plotting
    plt.plot(w, eta2, c='k', ls=lines[i], label='n = {}'.format(int(n)))

# Plot formatting
plt.xlim(0.8, 1.2)
plt.ylim(-0.1, 1.1)
plt.xticks([0.8, 0.9, 1.0, 1.1, 1.2])
plt.tick_params(axis='both', which='major', labelsize=10)
plt.xlabel('Fastness Parameter, $\Omega$', fontsize=12)
plt.ylabel(r'Propeller Efficiency, $\eta_2$', fontsize=12)
plt.legend(loc='lower right', fontsize=10);  # Legend added for clarity here - not present in the paper